In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
r = requests.get('https://malaysiaindru.my/?cat=1')

In [4]:
soup = BeautifulSoup(r.content, "lxml")

In [21]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if '?cat=' in a_.get('href')]
a

['/?cat=1', '/?cat=4', '/?cat=6', '/?cat=1033', '/?cat=9', '/?cat=3218']

In [19]:
a = soup.find_all('a')
[a_.get('href') for a_ in a if '/category' in a_.get('href')][0]

'https://malaysiaindru.my/category/news'

In [14]:
page_num = soup.find_all('a', {'class': "page-numbers"})
page_num = [a_.get('href', '').split('/page/')[-1] for a_ in page_num if '/page/' in a_.get('href', '')]
if len(page_num):
    page_num = max([int(i) for i in page_num])
else:
    page_num = 2
page_num

1204

In [17]:
soup.find_all('a', {'class': "page-numbers"})

[<a class="page-numbers" href="https://malaysiaindru.my/category/news/page/2">2</a>,
 <a class="page-numbers" href="https://malaysiaindru.my/category/news/page/1204">1,204</a>,
 <a class="next page-numbers" href="https://malaysiaindru.my/category/news/page/2"><i aria-hidden="true" class="fa fa-angle-right"></i></a>]

In [36]:
!rm -rf url
!mkdir url

In [38]:
for a_ in a:
    print(a_)
    r = requests.get(f'https://malaysiaindru.my{a_}')
    soup = BeautifulSoup(r.content, "lxml")
    url = [a_.get('href') for a_ in soup.find_all('a') if '/category' in a_.get('href')][0]
    page_num = soup.find_all('a', {'class': "page-numbers"})
    page_num = [a_.get('href', '').split('/page/')[-1] for a_ in page_num if '/page/' in a_.get('href', '')]
    if len(page_num):
        page_num = max([int(i) for i in page_num])
    else:
        page_num = 2
        
    for i in tqdm(range(1, page_num, 1)):
        
        filename = os.path.join('url', f"{a_.replace('/','')}-{i}.json")
        if os.path.exists(filename):
            continue
            
        new_url = f'{url}/page/{i}'
        r = requests.get(new_url)
        soup = BeautifulSoup(r.content, "lxml")
        articles = soup.find_all('article')
        articles = [article.find('a').get('href') for article in articles]
        with open(filename, 'w') as fopen:
            json.dump(articles, fopen)

/?cat=1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1203/1203 [00:00<00:00, 396709.47it/s]


/?cat=4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:36<00:00,  1.33it/s]


/?cat=6


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 362/362 [06:29<00:00,  1.07s/it]


/?cat=9


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:25<00:00,  1.56it/s]


/?cat=3218


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.06it/s]
